# Peak shaving und Optimierung

DThe power grid is designed to provide a reliable service. This often means that components are dimensioned to support the highest anticipated demand even if this occurs rarely. High load power peaks are undesirable, since they lead to oversized expensive infrastructure to avoid grid disruptions.

To persuade big consumers to reduce their peak power, their electricity bill includes, besides the regular energy consumption costs, charges for the maximum power measured during the billing period.
The consumers could reduce costs by adapting their consumption behavior, accommodating more flexible loads to low consumption times.
An alternative if this flexibility is not available, is to introduce a battery storage system to shift the loads and perform peak-shaving.
Overall the load profile will be flattened, while the consumption remains the same..

In this notebook we will:
* Explore peak-shaving with energy storage as a way to reduce power peak costs.
* Use optimization models to determine the peak-shaving strategy.
* Extend our optimization models to size our storage system.

In [ ]:
import pandas as pd
import pyomo.environ as opt
import plotly.graph_objects as go

In [ ]:
pd.options.plotting.backend = "plotly"
template = "plotly_white"
# template = "plotly_dark"

## Use case - dataset and base-scenario definition
For our example we take an historic load profile from HS Kempten Campus. For simplicity, we will consider a single billing period of 1 year.

In [ ]:
# use 15 min time step:
# optimization time:
#   task 1:     25s
#   task 2:     3m11s * 2
# profile =df_15min

# use 1h time step:
# optimization time:
#   task 1:     5s
#   task 2:     28s * 2 
dt = 1 # h

if dt == 0.25:    
    # load profile: version: 1h / 15 min time step version
    profile = pd.read_csv("..\data\HSK\HSK_22_15min.csv", index_col=['time_date'], parse_dates=True).drop(columns=['Unnamed: 0','time']).rename(columns={'pUser': 'load'})

if dt == 1:
    # load profile: version: 1h / 15 min time step version
    profile = pd.read_csv("..\data\HSK\HSK_22_1h.csv", index_col=0, parse_dates=True).rename(columns={'pUser': 'load'})


In [ ]:
profile.plot(template=template, labels={"value": "Power [kW]"})


<div class="alert alert-block alert-info">
Calculate energy consumption, peak power and the resulting electricity bill.
</div>

In [ ]:
# cost
electricity_cost =   0.12 # €/kWh
peak_power_cost  = 120.00 # €/kW

In [ ]:
# Calculate energy consumption, peak power and the resulting electricity bill.
def baseline_analysis(profile, electricity_cost, peak_power_cost):
    demand = profile.sum() * 0.25 # conversion from power (15 mins) to energy (per hour) 
    peak = profile.max()
    cost = demand * electricity_cost + peak * peak_power_cost
    print(f"Consumption: {demand:10.2f} kWh")
    print(f"Peak power:  {peak:10.2f} kW")
    print(f"Total costs: {cost:10.2f} €")

In [ ]:
print("--- without storage ---")
baseline_analysis(profile["pRes"], electricity_cost, peak_power_cost)

## Optimization model - Operation

An energy storage system can reduce costs by performing peak shaving. But how exactly should it operate? 

We can define a target maximum power peak and charge or discharge the storage depending on if the load is currently above or below this threshold.
Defining a peak target could be nonetheless difficult, if the threshold is too high the storage system may not be used at its full potential, if it is too low, the storage system would not be able to fulfil it.

We can instead use mathematical optimization techniques to find the optimal storage charge/discharge schedule to best fulfill the task. The [*pyomo* library](https://www.pyomo.org/) allows us to easily formulate *linear programming* models with python.

<div class="alert alert-block alert-info">
Formulation and optimization of the model
<ol>
    <li> Formulate the optimization problem in a markdown cell using \( \LaTeX \). </li>
    <li> Create a function <code>build_model</code> that takes storage system parameters <code>storage_params</code>, load profile <code>load_profile</code>, electricity costs <code>electricity_cost</code>, peak costs <code>peak_power_cost</code> and returns a pyomo optimization model <code>model</code>. </li>
    <li> Build the model based on the given storage parameters and optimize it with the GLPK solver. </li>
    <li> Recover the optimization results. Combine the time series results togther in a dataframe with the load profile. Make the following plots:
    <ol>
        <li> Load profile and grid power in the same plot. </li>
        <li> Storage power. </li>
        <li> Storage energy content or SOC. </li>
    </ol>
    </li>
    <li> Calculate the new total consumption, peak-power and total costs. Evaluate the improvements against the scenario without storage system. </li>
    
</ol>
</div>

$$
\begin{aligned}
&\min \sum_{T}(p^{grid,buy}_t \cdot \Delta t) \cdot k^{electricity} + p^{peak} \cdot k^{peak} \\
s.t.  &  \\
0 &\leq p^{ch}_t, \leq p^{N} \quad &\forall t \in T \\
0 &\leq p^{dch}_t, \leq p^{N} \quad &\forall t \in T \\
E^N \cdot SOC^{min} &\leq E^{bess}_t \leq E^N \cdot SOC^{max} \quad &\forall t \in T \\
E^{bess}_t &= E^{bess}_{t-1} + p^{ch}_t \Delta t \cdot \eta^{ch} - p^{dch}_t \Delta t \cdot (1/\eta^{dch}) \quad &\forall t \in T \\
E^{bess}_{t=0} &= E^{N} \cdot SOC^{init} \\ 
E^{bess}_{t=end} &\geq E^{N} \cdot SOC^{init} \\ 
p^{grid,buy}_t + p^{dch}_t &= p^{load}_t + p^{ch}_t \quad &\forall t \in T \\
p^{grid,buy}_t &\geq 0 \quad &\forall t \in T \\
p^{peak} &\geq 0  \\
p^{grid,buy}_t &\leq p^{peak} \quad &\forall t \in T \\
\end{aligned}
$$

**Variables**: Energy charge state at $t$ timestep $E^{bess}$, charge power at $t$ timestep $p^{ch}_t$, discharge power at $t$ timestep $p^{ch}_t$, grid power supply at $t$ timestep $p^{grid,buy}_t$, peak power $p^{peak}$.

**Parameters**: Storage system installed capacity $E^{N}$, storage system installed power $p^{N}$, electricity cost $k^{electricity}$, peak power cost $p^{peak}$, lower SOC limit $SOC^{min}$, upper SOC limit $SOC^{min}$, start SOC value $SOC^{init}$, charge efficiency $\eta^{ch}$, discharge efficiency $\eta^{dch}$, power demand at $t$ timestep $p^{load}_t$.

In [ ]:
# Storage parameters - Definition as "Dictionary"
storage_params = {
    "capacity": 200.0,  # kWh
    "power": 200.0,      # kW

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.1,

    "effc": 0.9,     # charge efficiency
    "effd": 0.9,     # discharge efficiency
}

In [ ]:
storage_params["effc"]

In [ ]:
# Build the model based on the given storage parameters and optimize it with the GLPK solver.
def build_model(storage_params, load_profile, electricity_cost, peak_power_cost):
    # Optimization model
    model = opt.ConcreteModel()
    ## =====  Parameters  ===== ##
    n = len(load_profile)   # number of timesteps
    model.time = opt.RangeSet(0, n-1) # T
    capacity  = storage_params["capacity"]
    max_power = storage_params["power"]
    
    soc_min, soc_max = storage_params["soc_bounds"]
    soc_start = storage_params["soc_start"]

    effc  = storage_params["effc"]
    effd  = storage_params["effd"]


    ## =====  Variables  ===== ##
    # BESS: power charge/discharge, energy content
    model.power_charge    = opt.Var(model.time, bounds=(0.0, max_power))
    model.power_discharge = opt.Var(model.time, bounds=(0.0, max_power))
    model.energy_bess     = opt.Var(model.time, bounds=(soc_min * capacity, soc_max * capacity))

    # Grid: power, peak-power limit (for peak-shaving)
    model.grid_power = opt.Var(model.time, within=opt.NonNegativeReals)
    model.peak_power = opt.Var(within=opt.NonNegativeReals)
    ## =====  Objective  ===== ##
    model.obj = opt.Objective(sense=opt.minimize, rule=sum(model.grid_power) * dt * electricity_cost + model.peak_power * peak_power_cost)

    ## ===== Constraints ===== ##
    # SOC balance
    def energy_balance_rule(m, t):
        if t == m.time.first(): #
            return m.energy_bess[t] == soc_start * capacity + dt * (effc * m.power_charge[t] - (1/effd) * m.power_discharge[t])
        return m.energy_bess[t] == m.energy_bess[t-1] + dt * (effc * m.power_charge[t] - (1/effd) * m.power_discharge[t])
    
    model.constraint_energy_balance = opt.Constraint(model.time, rule=energy_balance_rule)

    # power balance
    def power_balance_rule(m, t):
        return m.grid_power[t] + m.power_discharge[t] - m.power_charge[t] == load_profile.pRes[t]

    model.constraint_power_balance = opt.Constraint(model.time, rule=power_balance_rule)

    # peak
    def peak_power_rule(m, t):
        return m.grid_power[t] <= m.peak_power
    
    model.constraint_peak_power = opt.Constraint(model.time, rule=peak_power_rule)
    return model

In [ ]:
#  build model
model = build_model(
    storage_params=storage_params, 
    load_profile=profile, 
    electricity_cost=electricity_cost, 
    peak_power_cost=peak_power_cost
)

In [ ]:
# model.pprint()

In [ ]:
solver = opt.SolverFactory('glpk')  # glpk is an open source LP solver

In [ ]:
status = solver.solve(model, tee=True)

In [ ]:
opt.value(model.power_charge[1])

In [ ]:
# recover results
def recover_results(model, profile):
    df = profile.copy() # Solutions dataframe
    df["BESS_power"] = [opt.value(model.power_charge[t]) - opt.value(model.power_discharge[t]) for t in model.time]
    df["BESS_energy"] = [opt.value(model.energy_bess[t]) for t in model.time]
    df["grid_power"] = [opt.value(model.grid_power[t]) for t in model.time]
    return df

In [ ]:
# plot results 

def plot_storage_power(profile, model):
    powerc = [opt.value(model.power_charge[t]) for t in model.time]
    powerd = [- opt.value(model.power_discharge[t]) for t in model.time]
    
    fig = go.Figure()
    fig.update_layout(template=template)
    fig.add_trace(go.Scatter(x=profile.index, y=powerc, name="Power charge"))
    fig.add_trace(go.Scatter(x=profile.index, y=powerd, name="Power discharge"))
    fig.update_yaxes(title="Power [kW]")
    fig.update_traces(line_shape="hv")
    return fig

In [ ]:
# PLOT: residual Load profile and grid power in the same plot.
solution = recover_results(model, profile)

solution[["pRes", "grid_power"]].plot(template=template, labels={"value": "Power [kW]"})

In [ ]:
# PLOT:  Storage power. 
plot_storage_power(profile, model)

In [ ]:
# PLOT:  Storage energy content or SOE (i.e. State of Energy).
solution["BESS_energy"].plot(template=template, labels={"value": "Energy [kWh]"})

In [ ]:
def analyze_results(df, electricity_cost, peak_power_cost):
    peak = df["pRes"].max()
    demand = df["pRes"].sum() * 0.25
    cost = demand * electricity_cost + peak * peak_power_cost
    
    opt_peak = df["grid_power"].max()
    opt_demand = df["grid_power"].sum() * 0.25
    opt_cost = opt_demand * electricity_cost + opt_peak * peak_power_cost
    
    delta_cost = (opt_cost - cost)/cost
    delta_peak = (opt_peak - peak)/peak
    delta_demand = (opt_demand - demand)/demand
    
    print(f"Consumption: {opt_demand:10.2f} kWh ({delta_demand:+7.2%})")
    print(f"Peak power:  {opt_peak:10.2f} kW  ({delta_peak:+7.2%})")
    print(f"Total annual costs: {opt_cost:10.2f} €   ({delta_cost:+7.2%})")


In [ ]:
analyze_results(solution, electricity_cost, peak_power_cost)

<div class="alert alert-block alert-success">
Size optimization of the BESS


## Optimized system dimensioning

We have succesfully optimized our storage operation to minimizes costs. This was done with a predefined system. But what if we find ourselfs on the planning stage, contemplating the idea of aquiring a storage system to perform peak shaving? The storage system comes with respective investment costs, so me might want to 
re-formulate the optimization problem with a further degree of freedom to find the best system size that minimizes the total costs.

We consider a Storage system with costs and efficiency shown in the following table:

|                                   | Battery Parameters |
|:---------------------------------:|:-----------------:|
|  capacity costs           | 100.00 €/kWh      |
|  power costs              | 20.00 €/kW       |
| Efficiency charge ; discharge     | 95% ; 95%        |
|Expected service duration | 5 years|

<div class="alert alert-block alert-info">
<b>Task!</b> Formulate and optimize the dimensioning model
<ol>
    <li> Formulate the optimal sizing problem in a markdown cell using \( \LaTeX \). </li>
    <li> Create a function <code>build_dimension_model</code> that takes storage system parameters <code>storage_params</code>, load profile <code>load_profile</code>, electricity costs <code>electricity_cost</code>, peak costs <code>peak_power_cost</code> and returns a pyomo optimization model <code>model</code>. </li>
    <li> Build the model based on the given storage parameters and optimize it with the GLPK solver. </li>
    <li> Recover the optimization results. Combine the load profile and the time series results of both storage systems in a dataframe. Make the following plots:
    <ol>
        <li> Load profile and grid power in the same plot. </li>
        <li> Storage power. </li>
        <li> Storage energy content or SOC. </li>
    </ol>
    </li>
    <li> Calculate the new total consumption, peak-power and total costs for both systems. Which system would you choose? Give your arguments in a markdown cell. </li>
    
</ol>
</div>

TASK 1: LATEX


$$
\begin{aligned}
&\min \sum_{T}(p^{grid,buy}_t \cdot \Delta t) \cdot k^{electricity} + p^{peak} \cdot k^{peak} + p^{N} \cdot k^{inv,P} + E^{N} \cdot k^{inv, E} \\
s.t.  &  \\
0 &\leq p^{ch}_t, \leq p^{N} \quad &\forall t \in T \\
0 &\leq p^{dch}_t, \leq p^{N} \quad &\forall t \in T \\
E^N \cdot SOC^{min} &\leq E^{bess}_t \leq E^N \cdot SOC^{max} \quad &\forall t \in T \\
E^{bess}_t &= E^{bess}_{t-1} + p^{ch}_t \Delta t \cdot \eta^{ch} - p^{dch}_t \Delta t \cdot (1/\eta^{dch}) \quad &\forall t \in T \\
E^{bess}_{t=0} &= E^{N} \cdot SOC^{init} \\ 
E^{bess}_{t=8760} &\geq E^{N} \cdot SOC^{init} \\ 
p^{grid,buy}_t + p^{dch}_t &= p^{load}_t + p^{ch}_t \quad &\forall t \in T \\
p^{grid,buy}_t &\geq 0 \quad &\forall t \in T \\
p^{peak} &\geq 0  \\
p^{grid,buy}_t &\leq p^{peak} \quad &\forall t \in T \\
\end{aligned}
$$

**Variables**: Storage system installed capacity $E^{N}$, storage system installed power $p^{N}$, energy charge state at $t$ timestep $E^{bess}$, charge power at $t$ timestep $p^{ch}_t$, discharge power at $t$ timestep $p^{ch}_t$, grid power supply at $t$ timestep $p^{grid,buy}_t$, peak power $p^{peak}$.

**Parameters**: Energy capacity specific investment costs $k^{inv,E}$, power specific investment costs $k^{inv,E}$, electricity cost $k^{electricity}$, peak power cost $p^{peak}$, lower SOC limit $SOC^{min}$, upper SOC limit $SOC^{min}$, start SOC value $SOC^{init}$, charge efficiency $\eta^{ch}$, discharge efficiency $\eta^{dch}$, power demand at $t$ timestep $p^{load}_t$.


In [ ]:
operation_time =5 # years

In [ ]:
# Storage parameters

storage1_params = {
    "capacity_cost": 100.00/operation_time,  # €/kWh
    "power_cost": 20.00/operation_time,     # €/kW

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.0,

    "effc": 0.95,     # charge efficiency
    "effd": 0.95,     # discharge efficiency
}


In [ ]:
def build_dimension_model(storage_params, load_profile, electricity_cost, peak_power_cost):
    # Optimization model
    model = opt.ConcreteModel()

    ## =====  Parameters  ===== ##
    n = len(load_profile)   # number of timesteps
    model.time = opt.RangeSet(0, n-1) # T
    dt = 0.25               # 15 min timesteps
    
    capacity_cost = storage_params["capacity_cost"]
    power_cost = storage_params["power_cost"]
    
    soc_min, soc_max = storage_params["soc_bounds"]
    soc_start = storage_params["soc_start"]

    effc  = storage_params["effc"]
    effd  = storage_params["effd"]

    ## =====  Variables  ===== ##
    # BESS: power charge/discharge, energy content
    model.power_charge    = opt.Var(model.time, bounds=(0.0, None))
    model.power_discharge = opt.Var(model.time, bounds=(0.0, None))
    model.energy_bess     = opt.Var(model.time)

    # BESS size
    model.bess_capacity = opt.Var(within=opt.NonNegativeReals)
    model.bess_power    = opt.Var(within=opt.NonNegativeReals)

    # Grid: power, peak-power limit (for peak-shaving)
    model.grid_power = opt.Var(model.time, within=opt.NonNegativeReals)
    model.peak_power = opt.Var(within=opt.NonNegativeReals)


    ## =====  Objective  ===== ##
    def objective_rule(m):
        return sum(m.grid_power) * dt * electricity_cost + m.peak_power * peak_power_cost + m.bess_capacity * capacity_cost + m.bess_power * power_cost
    
    model.obj = opt.Objective(sense=opt.minimize, rule = objective_rule)


    ## ===== Constraints ===== ##
    # Power limits
    def power_charge_limit_rule(m, t):
        return m.power_charge[t] <= m.bess_power
    
    model.constraint_powerc_limit = opt.Constraint(model.time, rule=power_charge_limit_rule)
    
    def power_discharge_limit_rule(m, t):
        return m.power_discharge[t] <= m.bess_power

    model.constraint_powerd_limit = opt.Constraint(model.time, rule=power_discharge_limit_rule)

    # Energy content limits
    def energy_max_rule(m, t):
        return m.energy_bess[t] <= m.bess_capacity * soc_max

    model.constraint_energy_max = opt.Constraint(model.time, rule=energy_max_rule)

    def energy_min_rule(m, t):
        return m.energy_bess[t] >= m.bess_capacity * soc_min

    model.constraint_energy_min = opt.Constraint(model.time, rule=energy_min_rule)

    # SOC balance
    def energy_balance_rule(m, t):
        if t == m.time.first(): #
            return m.energy_bess[t] == soc_start * m.bess_capacity + dt * (effc * m.power_charge[t] - (1/effd) * m.power_discharge[t])
        return m.energy_bess[t] == m.energy_bess[t-1] + dt * (effc * m.power_charge[t] - (1/effd) * m.power_discharge[t])
    
    model.constraint_energy_balance = opt.Constraint(model.time, rule=energy_balance_rule)

    # SOC end conditions
    def energy_end_rule(m):
        return m.energy_bess[m.time.last()] >= soc_start * m.bess_capacity
    
    model.constraint_energy_end = opt.Constraint(rule=energy_end_rule)

    # power balance
    def power_balance_rule(m, t):
        return m.grid_power[t] + m.power_discharge[t] - m.power_charge[t] == load_profile.pRes[t]

    model.constraint_power_balance = opt.Constraint(model.time, rule=power_balance_rule)

    # peak
    def peak_power_rule(m, t):
        return m.grid_power[t] <= m.peak_power
    
    model.constraint_peak_power = opt.Constraint(model.time, rule=peak_power_rule)

    return model

In [ ]:
model_dim1 = build_dimension_model(
    storage_params=storage1_params,
    load_profile=profile, 
    electricity_cost=electricity_cost, 
    peak_power_cost=peak_power_cost
)
status = solver.solve(model_dim1, tee=True)

In [ ]:
# task: recover results
res_dim1 = recover_results(model_dim1, profile)


In [ ]:
res_dim1.head()

In [ ]:
# task: analyze results
def analyze_dimension_results(df, model, storage_params, electricity_cost, peak_power_cost):
    # system dimension
    capacity = opt.value(model.bess_capacity)
    power    = opt.value(model.bess_power)
    
    capacity_cost = storage_params["capacity_cost"]
    power_cost = storage_params["power_cost"]
    system_cost = capacity * capacity_cost*operation_time + power * power_cost*operation_time
    
    # operation
    peak = df["pRes"].max()
    demand = df["pRes"].sum() * 0.25
    annual_cost = demand * electricity_cost + peak * peak_power_cost
    
    opt_peak = df["grid_power"].max()
    opt_demand = df["grid_power"].sum() * 0.25
    annual_opt_cost = opt_demand * electricity_cost + opt_peak * peak_power_cost + system_cost
    
    delta_cost = (annual_cost - annual_opt_cost)/annual_opt_cost
    delta_peak = (opt_peak - peak)/peak
    delta_demand = (opt_demand - demand)/demand
    
    print(f"Installed capacity: {capacity:7.2f} kWh")
    print(f"Installed power:    {power:7.2f} kW")
    print(f"System costs:       {system_cost:7.2f} €\n")
    
    print(f"annual Consumption: {opt_demand:10.2f} kWh ({delta_demand:+7.2%})")
    print(f"Peak power:  {opt_peak:10.2f} kW  ({delta_peak:+7.2%})")
    print(f"Total costs: {annual_opt_cost*operation_time:10.2f} €   ({delta_cost:+7.2%})")



In [ ]:
# task: analyze resuls
print("--- storage 1 ---")
analyze_dimension_results(res_dim1, model_dim1, storage1_params, electricity_cost, peak_power_cost)
print()

In [ ]:
res_dim1.loc[:, ["pRes", "grid_power"]].plot(template=template)

In [ ]:
res_dim.loc[:, ["BESS_power_1"]].plot(template=template)